# 출력 파서(`OutputParser`)

LLM의 출력을 더 유용, 구조화된 형태로 변환함

1. 구조화: LLM의 자유분방한 형식의 출력을 JSON, dict등의 형태로 **변환**할 수 있다. -> 쓰기 편함
2. 일관성: 항상 일관된 출력형식을 주기 때문에, 후속 처리에 용이함
3. 유연성: 다양한 출력 형식이 존재. JSON, list, dict 등등 다 가능함

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()
llm = ChatOpenAI(model='gpt-4.1-nano')

## PydanticOutputParser

파이썬 객체로 볼 수 있게 바꿔주는 편

JSON화는 JsonOutputParser가 따로 있긴 함

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [11]:
# 일반적인 이메일 형식, input
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [20]:
prompt = PromptTemplate.from_template("""
아래 이메일에서 핵심 정보를 추출해 줘
---
{email_conversation}
""")

print(prompt.format(email_conversation=email_conversation))


아래 이메일에서 핵심 정보를 추출해 줘
---
From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션




In [21]:
chain = prompt | llm
res = chain.invoke({'email_conversation': email_conversation}).content
print(res)  # 사람용 요약으로는 훌륭하지만, 구조화된 답변은 아님. 모든 이메일을 정해진 포맷에 맞게 정리하고 싶다면?

아래는 이메일의 핵심 정보입니다:

1. 발신인: 김철수 상무(바이크코퍼레이션)
2. 수신인: 이은채 대리(티디인터내셔널)
3. 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. 요청 내용:
   - "ZENESIS" 모델의 상세 브로슈어 요청
   - 포함 요청 항목: 기술 사양, 배터리 성능, 디자인 정보
5. 미팅 일정:
   - 날짜 및 시간: 1월 15일(화요일) 오전 10시
   - 장소: 귀사(티디인터내셔널) 사무실
6. 목적:
   - 유통 전략 및 마케팅 계획 구체화
   - 협력 가능성 논의


In [32]:
# Field의 description을 LLM이 읽어서 이해한다
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='메일 보낸 사람의 메일 주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')
    
parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())  # LLM에게 주는 지침

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일 보낸 사람의 메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [31]:
prompt = PromptTemplate.from_template("""
너는 요약 전문 어시스턴트야. 아래 질문과 형식에 맞게 답변을 한국어로 만들어줘.

---

질문: {question}

이메일 내용: {email_conversation}

형식: {format}

""")

# 일부 고정 변수
prompt = prompt.partial(format=parser.get_format_instructions())

In [35]:
chain = prompt | llm | parser
res = chain.invoke(
    {
        'question': '이메일 내용 중에 중요한 내용을 추출해 줘',
        'email_conversation' : email_conversation
    }
    )

In [ ]:
from pprint import pprint
pprint(res.model_dump())
# 앞으로 어떤 메일을 넣어도 지금처럼 형식을 맞춰 줄거라는 것

{'date': '2024년 1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수 상무',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션 김철수 상무는 귀사의 'ZENESIS' 자전거에 대해 상세 브로슈어 요청과 함께, 기술 사양, 배터리 "
            '성능, 디자인 정보 확보를 희망하며, 다음 주 1월 15일 화요일 오전 10시에 사무실에서 협력 논의를 위한 미팅을 '
            '제안함.'}


## CSV Parser  

In [52]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='List 5 {subject}. \n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

prompt.format(subject='중국집 대표메뉴')

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


'List 5 중국집 대표메뉴. \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [59]:
chain_short = prompt | llm 
chain_parser = prompt | llm | output_parser


csv_res = chain_short.invoke({'subject': '맥도날드 대표메뉴'}).content
lst_res = chain_parser.invoke({'subject': '대한민국의 국민주식'})

In [ ]:
print(csv_res)  # csv같은 형태로 (사람이)보기 위한 방법
print(lst_res)  # 리스트로 csv 파일 만들기 활용도가 커서 보통 이렇게 만든다

Big Mac, Quarter Pounder, Chicken McNuggets, McChicken, Fries
['삼성전자', 'SK하이닉스', 'LG전자', '현대차', 'POSCO']


## Structured Output Parser

dict 형식 -> 멍청한 모델에도 적용 가능함

In [71]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser  # 이건 langchain_core가 아니라 langchain에 있음

response_schemas = [
    ResponseSchema(name='answer', description='사용자의 질문에 대한 답변'),  # type도 정하는게 가능한데, 꼭 정해진 문자가 아니라 LLM이 알아들을 수만 있으면 됨
    ResponseSchema(name='source', description='질문에 답하기 위해 사용된 웹사이트 주소 출처')
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser.get_format_instructions())  # 진짜 json으로 나옴

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 웹사이트 주소 출처
}
```


In [ ]:
prompt = PromptTemplate(
    template='사용자 질문에 최선을 다해 답변함.\n질문: {question}\n{format_instructions}',
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

print(prompt.format(question='처서는 몇월 몇일인가?'))

사용자 질문에 최선을 다해 답변함.
질문: 처서는 몇월 몇일인가?
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 웹사이트 주소 출처
}
```


In [77]:
chain = prompt | llm | output_parser
res = chain.invoke({'question':'처서는 몇월 몇일인가?'})
pprint(res)

# stream을 해서 토큰별로 확인하는 건 안됨 - 이미 후처리(딕셔너리 형태로 변환)한 답변이기 떄문

{'answer': '처서(處暑)는 음력 24번째 절기로, 양력으로 보통 8월 23일경에 해당합니다. 정확한 날짜는 매년 달라질 수 '
           '있습니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%B2%98%EC%84%9C'}


## DataFrame Output Parser

원하는 데이터프레임을 분석하고 데이터프레임으로 답변도 받을 수 있다

In [135]:
from langchain.output_parsers import PandasDataFrameOutputParser
import pandas as pd
import seaborn as sns

titanic_df = sns.load_dataset('titanic')

output_parser = PandasDataFrameOutputParser(dataframe=titanic_df)  # 분석 대상인 데이터프레임을 마지막에 제공하는 셈 -> 컬럼 목록을 제공하기 위해
print(output_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [136]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\nuser query: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [139]:
chain = prompt | llm | output_parser
res = chain.invoke({'query': 'age 컬럼을 조회해 줘'})
print(res)

{'age': 0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64}


In [140]:
res = chain.invoke({'query': 'age 컬럼의 평균을 조회해 줘'})
print(res)

{'mean': np.float64(29.69911764705882)}


In [142]:
# res = chain.invoke({'query': 'age 컬럼의 결측치를 전체 컬럼의 평균으로 채워줘'})
# print(res)
# 생각보다 복잡한건 잘 못함

## Datetime Output Parser

In [109]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser(format = '%Y-%m-%d')
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n',
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

Write a datetime string that matches the following pattern: '%Y-%m-%d'.

Examples: 2025-09-02, 2024-09-02, 2025-09-01

Return ONLY this string, no other words!


In [117]:
chain = prompt | llm  | output_parser
res = chain.invoke({'question': '내가 로또를 사면 당첨될 날은 언제일까?'})
print(res)

2024-04-27 00:00:00


## Enum Output Parser

Enumerate, 열거형 출력 파서

정해진 목록 중에 답을 고르는 것(객관식)

In [144]:
from langchain.output_parsers import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = '빨간색'
    GREEN = '초록색'
    BLUE = '파란색'
    WHITE = '하얀색'
    BLACK = '검은색'
    
output_parser = EnumOutputParser(enum=Colors)
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='다음 질문에 답해줘.\n\n#Instructions: {format_instructions} \n\n#질문: {object}는 어떤 색에 가장 가까울까?',
    input_variables=['object'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

Select one of the following options: 빨간색, 초록색, 파란색, 하얀색, 검은색


In [145]:
chain = prompt | llm | output_parser
res = chain.invoke({'object': '병아리'})
print(res)

Colors.GREEN


## Output Parser + Agent

Agent가 정해진 출력으로 나오게 하기

In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field

# 1. pydantic 클래스(답변 형태)
class Classification(BaseModel):
    sentiment: str = Field(description='작성된 글의 감정')
    agressiveness: int = Field(description='얼마나 공격적인지를 1~10점으로 판단')
    language: str = Field(description='작성된 글의 언어')
    remarks: str = Field(description='명사형으로 특이사항을 요약. 특이사항은 문제사항이나 문제가 있던 이유를 말하며, 특이사항이 없을 경우 작성하지 말 것')

# 2. LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
structured_llm = llm.with_structured_output(Classification)

# 3. 프롬프트
prompt = ChatPromptTemplate([
    ('system', '당신은 텍스트에서 감정, 공격성, 언어, 특이사항을 추출하는 분류기입니다.\n응답은 항상 한국어로 해야 합니다.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

# 4. 메모리
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

# 5. agent 조립
agent = create_openai_tools_agent(
    llm=llm,
    tools=[],
    prompt=prompt
)

# 6. Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[],
    memory=memory,
    verbose=True
)

from langchain_core.runnables import RunnableLambda

pipeline = (
    agent_executor 
    | RunnableLambda(lambda x: x['output'])
    | structured_llm
)

pipeline.invoke({'input': "La comida fue terrible. Llegó después de una hora y ya estaba fría. Además, faltaba uno de los platos que pedí y el sabor era pésimo. Si van a trabajar así, mejor no lo hagan."})



> Entering new AgentExecutor chain...
{
  "감정": "불만, 실망",
  "공격성": "높음",
  "언어": "부정적, 비판적",
  "특이사항": "음식 품질 불만, 서비스 지연, 주문 누락"
}

> Finished chain.


Classification(sentiment='불만, 실망', agressiveness=8, language='부정적, 비판적', remarks='고객이 음식 품질, 서비스 지연, 주문 누락에 대해 강한 불만과 실망을 표현하고 있으며, 공격성이 높아 주의가 필요합니다.')